In [178]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, max_error
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer, RobustScaler

In [180]:
data = pd.read_csv('cleaneddata.csv')
data['underweight'] = [1 if x < 18 else 0 for x in data['bmi']]
data['overweight'] = [1 if x > 31 and x <= 38 else 0 for x in data['bmi']]
data['severlyoverweight'] = [1 if x > 38 else 0 for x in data['bmi']]
data['nokid'] = [1 if x ==0 else 0 for x in data['children']]
data['onekid'] = [1 if x ==1 else 0 for x in data['children']]
data['twokid'] = [1 if x ==2 else 0 for x in data['children']]
data['threekid'] = [1 if x ==3 else 0 for x in data['children']]
data['fourkid'] = [1 if x ==4 else 0 for x in data['children']]
data['fivekid'] = [1 if x ==5 else 0 for x in data['children']]
data['kids'] = [1 if x in [1,2,3,4] else 0 for x in data['children']]
data['elderfemale'] = [1 if x > 60 else 0 for x in data['age']]

In [181]:
X = data[['smoker','age','is_east','bmi', 'is_north','underweight', 'overweight','severlyoverweight', 'nokid','onekid','twokid','threekid', 'fourkid', 'fivekid','sex']]
y = data['charges']

In [183]:
X['bmi'] = (X['bmi'] - X['bmi'].mean())/X['bmi'].std()
X['age'] = (X['age'] - X['age'].mean())/X['age'].std()
# Q1 = X['bmi'].quantile(0.25)
# Q3 = X['bmi'].quantile(0.75)
# IQR = Q3 - Q1
# Q12 = X['age'].quantile(0.25)
# Q32 = X['age'].quantile(0.75)
# IQR2 = Q32 - Q12
# X['bmi'] = (X['bmi'] - X['bmi'].median())/IQR
# X['age'] = (X['age'] - X['age'].median())/IQR2
# X['bmi'] = (X['bmi'] - X['bmi'].median())/X['bmi'].std()
# X['age'] = (X['age'] - X['age'].median())/X['age'].std()

/tmp/ipykernel_4980/1688094482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['bmi'] = (X['bmi'] - X['bmi'].mean())/X['bmi'].std()
/tmp/ipykernel_4980/1688094482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age'] = (X['age'] - X['age'].mean())/X['age'].std()


In [6]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.LinearRegression()

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print(max_error(y_test,y_pred))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

4614.374609619625
Coefficient of determination: 0.84
18204.815658396852
root mean squared error: 4074.32
Coefficient of determination: 0.91


In [82]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

regression = linear_model.Lasso(alpha=3.9, tol=0.01)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("root mean squared error: %.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.4f" % r2_score(y_test, y_pred))

root mean squared error: 3416.923
Coefficient of determination: 0.9194


In [129]:
test = np.arange(0, 1, 0.05)
for i in test:
    data['overweight'] = [1 if x > (31+i) and x <= (38+i) else 0 for x in data['bmi']]
    data['severlyoverweight'] = [1 if x > (38+i) else 0 for x in data['bmi']]
    X = data[['smoker','age','is_east','bmi', 'is_north','underweight', 'overweight','severlyoverweight', 'nokid','kids', 'elderfemale']]
    y = data['charges']
    poly = PolynomialFeatures(degree=2)
    poly_variables = poly.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

    alpha_ranges = np.arange(4.1, 4.3, 0.001)
    maxscore = 0
    maxalp = 0
    for alp in alpha_ranges:
        regression = linear_model.Lasso(alpha=alp, tol=0.09)
        model = regression.fit(X_train, y_train)
        if model.score(X_test,y_test) > maxscore:
            maxscore = model.score(X_test,y_test)
            maxalp = alp
            para = [i]
print(maxalp, maxscore, para)

4.155000000000018 0.8984814811400483 [np.float64(0.9500000000000001)]


In [189]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

alpha_ranges = np.arange(0.1, 2, 0.01)
maxscore = 0
maxalp = 0
for alp in alpha_ranges:
    regression = linear_model.Lasso(alpha=alp, tol=0.07)
    model = regression.fit(X_train, y_train)
    if model.score(X_test,y_test) > maxscore:
        maxscore = model.score(X_test,y_test)
        maxalp = alp
print(maxalp, maxscore)

0.5399999999999998 0.9146344524998974


In [103]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

alpha_ranges = np.arange(0.001, 10, 0.0001)
maxscore = 0
maxalp = 0
for alp in alpha_ranges:
    regression = linear_model.Ridge(alpha=alp, tol=0.01)
    model = regression.fit(X_train, y_train)
    if model.score(X_test,y_test) > maxscore:
        maxscore = model.score(X_test,y_test)
        maxalp = alp
print(maxalp, maxscore)

1.2974000000000006 0.919084507557251


In [190]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

alpha_ranges = np.arange(1, 25, 1)
l1_range = np.arange(0.1, 1, 0.1)
maxscore = 0
maxalp = 0
maxl = 0
for alp in alpha_ranges:
    for l in l1_range:
        regression = linear_model.ElasticNet(alpha=alp, l1_ratio= l,tol=0.085)
        model = regression.fit(X_train, y_train)
        if model.score(X_test,y_test) > maxscore:
            maxscore = model.score(X_test,y_test)
            maxalp = alp
            maxl = l
print(maxalp, maxl, maxscore)

1 0.9 0.8814394181349192


In [89]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

regression = linear_model.Ridge(alpha=18, tol=0.01)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("root mean squared error: %.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.4f" % r2_score(y_test, y_pred))

root mean squared error: 3528.610
Coefficient of determination: 0.9141


In [93]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

regression = linear_model.ElasticNet(alpha=1.5, l1_ratio=0.89,tol=0.01)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("root mean squared error: %.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.4f" % r2_score(y_test, y_pred))

root mean squared error: 4754.837
Coefficient of determination: 0.8440


In [24]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

model = linear_model.ElasticNet(tol=0.09)

# test this range of alpha
alpha_range = np.arange(-2, 2, 0.01)
l1_range = np.arange(0.1,1,0.01)

param = {'alpha':alpha_range, 'l1_ratio': l1_range}

# score base on r2
grid_search_r2 = GridSearchCV(model,param_grid=param,cv=5, scoring='r2')
grid_search_r2.fit(X_train, y_train)


print('Scoring R2')
print('Best R2 score   : ', grid_search_r2.best_score_)
print('Best parameters : ', grid_search_r2.best_params_)

/home/david/Briefprimeassurance/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
90000 fits failed out of a total of 180000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "/home/david/Briefprimeassurance/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/david/Briefprimeassurance/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/home/david/Briefprimeassurance/.venv/lib/python3.12/site-packages/sklearn/base.py", line 436, in 

Scoring R2
Best R2 score   :  0.8301746882893266
Best parameters :  {'alpha': np.float64(0.020000000000001794), 'l1_ratio': np.float64(0.5099999999999998)}


In [37]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

model = linear_model.ElasticNetCV(n_alphas=1000)
model.fit(X_train, y_train)
print('Best parameters : ', model.alphas_)

Best parameters :  [7249.06355931 7199.11157712 7149.50380498 7100.23787101 7051.31141964
 7002.72211158 6954.46762363 6906.5456486  6858.95389521 6811.69008796
 6764.75196703 6718.13728819 6671.84382264 6625.86935696 6580.21169299
 6534.8686477  6489.83805311 6445.11775618 6400.7056187  6356.59951721
 6312.79734287 6269.29700136 6226.09641282 6183.1935117  6140.58624668
 6098.27258061 6056.25049033 6014.51796665 5973.07301423 5931.91365145
 5891.03791038 5850.44383663 5810.12948927 5770.09294077 5730.33227687
 5690.84559649 5651.63101166 5612.68664742 5574.01064174 5535.60114539
 5497.45632192 5459.5743475  5421.9534109  5384.59171336 5347.48746849
 5310.63890225 5274.04425279 5237.70177042 5201.60971751 5165.76636839
 5130.17000928 5094.81893823 5059.711465   5024.845911   4990.22060921
 4955.83390409 4921.68415153 4887.76971871 4854.0889841  4820.64033732
 4787.4221791  4754.43292118 4721.67098625 4689.13480788 4656.82283041
 4624.73350892 4592.86530912 4561.2167073  4529.78619026 4

In [34]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.ElasticNet(alpha=0.02, l1_ratio=0.51)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print(max_error(y_test,y_pred))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

10254.140927531776
Coefficient of determination: 0.23
45379.17243372946
root mean squared error: 11901.25
Coefficient of determination: 0.23


In [19]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.Lasso(alpha=0.999, tol=0.09)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print(max_error(y_test,y_pred))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

4725.553568513271
Coefficient of determination: 0.84
19647.613836337066
root mean squared error: 4237.41
Coefficient of determination: 0.90


In [60]:
poly = SplineTransformer(n_knots=4,degree=4)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.LinearRegression()

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))

print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

5938.189170785875
Coefficient of determination: 0.74
root mean squared error: 6178.33
Coefficient of determination: 0.79


In [61]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.Lasso(alpha=0.8, tol=0.08)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

4686.113077706826
Coefficient of determination: 0.84
root mean squared error: 4177.84
Coefficient of determination: 0.91


In [62]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.Ridge(alpha=0.1)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

4615.306346757694
Coefficient of determination: 0.84
root mean squared error: 4076.73
Coefficient of determination: 0.91


In [63]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.ElasticNet(alpha=0.031, l1_ratio=0.92, tol=0.08)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

4637.657562453623
Coefficient of determination: 0.84
root mean squared error: 4161.16
Coefficient of determination: 0.91


In [64]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.BayesianRidge(alpha_1=0.07, alpha_2= 0.07, lambda_1= 0.01, lambda_2=0.01)

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_predtrain = model.predict(X_train)

print(np.sqrt(mean_squared_error(y_train, y_predtrain)))
print("Coefficient of determination: %.2f" % r2_score(y_train, y_predtrain))
print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

4639.70178856866
Coefficient of determination: 0.84
root mean squared error: 4169.56
Coefficient of determination: 0.91


In [65]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.RANSACRegressor()

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

root mean squared error: 4427.72
Coefficient of determination: 0.89


In [66]:
poly = PolynomialFeatures(degree=2)
poly_variables = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_variables, y, test_size = 0.2, random_state = 42)

regression = linear_model.SGDRegressor()

model = regression.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

root mean squared error: 4250.36
Coefficient of determination: 0.90
